In [1]:
## -*- coding: utf-8 -*-
"""
Created on Wed Feb  6 20:41:00 2019

@author: tobi carvalho et simon dufort-labbé
"""


import os
import pickle
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
%matplotlib inline

print(f"Your version of Pytorch is {torch.__version__}. You should use a version >0.4.")


if torch.cuda.is_available():
    device = torch.device("cuda")
    use_cuda = True
else:
    device = torch.device("cpu")
    use_cuda = False
    
print(device)

Your version of Pytorch is 1.0.1.post2. You should use a version >0.4.
cuda


In [2]:
# On créer les dataset de MNIST

train_data = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ]))

test_data = datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ]))




Processing...
Done!


In [0]:
#On sélectionne au hasard qui est dans l'ensemble d'entraîement et qui est dans l'ensemble de validation
indices = list(range(len(train_data)))
random.shuffle(indices)



# On créer les DataLoader

batch_size = 100
batch_size_eval = 512
n_valid = 10000

train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(indices[n_valid:]),
    num_workers=1,
    pin_memory=use_cuda
)

valid_loader = DataLoader(
    train_data,
    batch_size=batch_size_eval,
    sampler=SubsetRandomSampler(indices[:n_valid]),
    num_workers=1,
    pin_memory=use_cuda,
)



In [0]:
# On définie les fonctions de pertes utilisé
loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='sum')


# On créer les fonctions pour l'entrainement et la validation sur une epoch

def train(model,train_loader, optimizer, epoch ):
    model.train()
    
    for batch_idx, (inputs, target) in enumerate(train_loader):
        inputs, target = inputs.to(device), target.to(device)
        
       
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(inputs), len(train_loader) *len(inputs) ,
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader):
    model.eval()
    
    test_loss = 0
    correct = 0
    test_size = 0
    with torch.no_grad():
        for inputs, target in test_loader:
            inputs, target = inputs.to(device), target.to(device)
           
            output = model(inputs)
            test_size += len(inputs)
            test_loss += test_loss_fn(output, target).item() 
            pred = output.max(1, keepdim=True)[1] 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= test_size
    accuracy = correct / test_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, test_size,
        100. * accuracy))
    
    return test_loss, accuracy

In [0]:
# Structure du CNN

class CNN(nn.Module):
    """A medium sized network that performs very well on MNIST."""
    
    def __init__(self):
        super().__init__()
        # conv block 1
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        # conv block 2
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        # fully connected layers
        self.fc1 = nn.Linear(64*7*7, 256)
        self.bn5 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        # [bs, 1, 28, 28]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2) 
        
        # [bs, 32, 14, 14]
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2) 
        
        # [bs, 64, 7, 7]
        x = x.view(x.size(0), -1) # flatten
        
        # x is [bs, 64x7x7]
        x = F.relu(self.bn5(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x
    

In [0]:

model = CNN().to(device)

lr = 0.005
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, 10):
    train(model, train_loader, optimizer, epoch)
    loss, acc = test(model, valid_loader)
    

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.391107
Train Epoch: 1 [1000/50000 (2%)]	Loss: 0.451279
Train Epoch: 1 [2000/50000 (4%)]	Loss: 0.216826
Train Epoch: 1 [3000/50000 (6%)]	Loss: 0.093974
Train Epoch: 1 [4000/50000 (8%)]	Loss: 0.032658
Train Epoch: 1 [5000/50000 (10%)]	Loss: 0.113841
Train Epoch: 1 [6000/50000 (12%)]	Loss: 0.088664
Train Epoch: 1 [7000/50000 (14%)]	Loss: 0.165094
Train Epoch: 1 [8000/50000 (16%)]	Loss: 0.193487
Train Epoch: 1 [9000/50000 (18%)]	Loss: 0.075882
Train Epoch: 1 [10000/50000 (20%)]	Loss: 0.074032
Train Epoch: 1 [11000/50000 (22%)]	Loss: 0.173701
Train Epoch: 1 [12000/50000 (24%)]	Loss: 0.072298
Train Epoch: 1 [13000/50000 (26%)]	Loss: 0.030829
Train Epoch: 1 [14000/50000 (28%)]	Loss: 0.123495
Train Epoch: 1 [15000/50000 (30%)]	Loss: 0.085109
Train Epoch: 1 [16000/50000 (32%)]	Loss: 0.021452
Train Epoch: 1 [17000/50000 (34%)]	Loss: 0.077387
Train Epoch: 1 [18000/50000 (36%)]	Loss: 0.030384
Train Epoch: 1 [19000/50000 (38%)]	Loss: 0.033503
Train Epoch: 1 [20